In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import resample_poly


In [ ]:
from pathlib import Path

In [ ]:
def iq_plot(i_signal, q_signal):
    plt.scatter(i_signal, q_signal)
    plt.title("IQ Plot")
    plt.xlabel("I")
    plt.ylabel("Q")
    plt.show()

def plot_power(signal, color="tab:blue", alpha=1.0):
    N = signal.shape[0]
    freq = np.arange(f/-2, f/2, f/N)
    fft_signal = np.fft.fft(signal)
    power_signal = 10*np.log10(np.fft.fftshift(np.absolute(fft_signal)))
    plt.plot(freq, power_signal, color=color, alpha=alpha)
    plt.title("Signal Power [dB]")
    plt.xlabel("freq")
    plt.ylabel("dB")
    #plt.show()

def get_spectogram(x):
    sample_rate = 5e6
    fft_size = 1024
    num_rows = len(x) // fft_size # // is an integer division which rounds down
    spectrogram = np.zeros((num_rows, fft_size))
    for i in range(num_rows):
        spectrogram[i,:] = 10*np.log10(np.abs(np.fft.fftshift(np.fft.fft(x[i*fft_size:(i+1)*fft_size])))**2)

    plt.imshow(spectrogram, aspect='auto', extent = [sample_rate/-2/1e6, sample_rate/2/1e6, 0, len(x)/sample_rate])
    plt.xlabel("Frequency [MHz]")
    plt.ylabel("Time [s]")
    plt.show()


def time_sync(x):
    samples = x # for the sake of matching the sync chapter
    samples_interpolated = resample_poly(samples, 32, 1) # we'll use 32 as the interpolation factor, arbitrarily chosen, seems to work better than 16
    sps = 2
    mu = 0.01 # initial estimate of phase of sample
    out = np.zeros(len(samples) + 10, dtype=np.complex64)
    out_rail = np.zeros(len(samples) + 10, dtype=np.complex64) # stores values, each iteration we need the previous 2 values plus current value
    i_in = 0 # input samples index
    i_out = 2 # output index (let first two outputs be 0)
    while i_out < len(samples) and i_in+32 < len(samples):
        out[i_out] = samples_interpolated[i_in*32 + int(mu*32)] # grab what we think is the "best" sample
        out_rail[i_out] = int(np.real(out[i_out]) > 0) + 1j*int(np.imag(out[i_out]) > 0)
        x = (out_rail[i_out] - out_rail[i_out-2]) * np.conj(out[i_out-1])
        y = (out[i_out] - out[i_out-2]) * np.conj(out_rail[i_out-1])
        mm_val = np.real(y - x)
        mu += sps + 0.01*mm_val
        i_in += int(np.floor(mu)) # round down to nearest int since we are using it as an index
        mu = mu - np.floor(mu) # remove the integer part of mu
        i_out += 1 # increment output index
    x = out[2:i_out] # remove the first two, and anything after i_out (that was never filled out)

    return x


# For QPSK
def phase_detector_4(sample):
    if sample.real > 0:
        a = 1.0
    else:
        a = -1.0
    if sample.imag > 0:
        b = 1.0
    else:
        b = -1.0
    return a * sample.imag - b * sample.real


def fine_freq_sync(x):
    # Fine freq sync
    samples = x # for the sake of matching the sync chapter
    N = len(samples)
    phase = 0
    freq = 0
    # These next two params is what to adjust, to make the feedback loop faster or slower (which impacts stability)
    alpha = 8e10
    beta = 2.2
    out = np.zeros(N, dtype=np.complex64)
    freq_log = []
    for i in range(N):
        out[i] = samples[i] * np.exp(-1j*phase) # adjust the input sample by the inverse of the estimated phase offset
        error = np.real(out[i]) * np.imag(out[i]) # This is the error formula for 2nd order Costas Loop (e.g. for BPSK)
        #error = phase_detector_4(out[i])

        # Advance the loop (recalc phase and freq offset)
        freq += (beta * error)
        freq_log.append(freq * sample_rate / (2*np.pi)) # convert from angular velocity to Hz for logging
        phase += freq + (alpha * error)

        # Optional: Adjust phase so its always between 0 and 2pi, recall that phase wraps around every 2pi
        # while phase >= 2*np.pi:
        #    phase -= 2*np.pi
        # while phase < 0:
        #    phase += 2*np.pi
    x = out
    return x, freq_log

# Load Data

In [ ]:
PATH = "data"

# Frequency= 2460 MHz
# RF Bandwidth = 25 MHz
# FILE = "capture_2460mhz"

# Frequency= 102.5 MHz
# RF Bandwidth = 5 MHz
FILE = "capture_102_5mhz"

# Frequency= 848 MHz
# RF Bandwidth = 10 MHz
#FILE = "capture_848mhz"

In [ ]:
path = Path(PATH)
data = np.loadtxt(path/FILE, skiprows=1)

# Data Conditioning

In [ ]:
# Transform data into complex format

# Create IQ components


In [ ]:
# Plot IQ signals


In [ ]:
# Sampling parameters

# Create a time vector


# Tansform IQ data into a single signal

# Received signal


In [ ]:
# Plot power


In [ ]:
# Plot spectogram


# Frequency Shift

In [ ]:
# Freq shift


In [ ]:
# Plot power of original and shifted signal


In [ ]:
# Plot spectogram


# Decimate Data

In [ ]:
# Decimate data by a factor of 10


In [ ]:
# Plot power of decimated signal


In [ ]:
# Plot spectogram


# Filter Data

In [ ]:
# Filter parameters

# create our low pass filter


In [ ]:
# plot the impulse response of filter


In [ ]:
# Plot spectogram


In [ ]:
# Filter decimated signal by use of convolution


In [ ]:
# Plot power of filtered signal


In [ ]:
# Plot spectogram


In [ ]:
# Plot IQ plot of filtered signal


# Time Synch

In [ ]:
# Apply time synchonization correction


In [ ]:
# Plot corrected IQ  signals
